In [ ]:
import numpy as np
import os

from tensorflow import keras
from tqdm import tqdm
import cv2 , pickle

In [ ]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive

drive.mount('/content/drive')

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


Mounted at /content/drive


In [ ]:
wd = './drive/My Drive/Colab Notebooks/DD2412_Project/'
local_download_path = os.path.expanduser(f'{wd}ILSVRC2012_img_val')
local_download_path

'./drive/My Drive/Colab Notebooks/DD2412_Project/ILSVRC2012_img_val'

In [ ]:
from google.colab import files
src_model = list(files.upload().values())[0]
src_utils = list(files.upload().values())[0]
src_expl = list(files.upload().values())[0]

open('model.py','wb').write(src_model)
open('image_utils.py','wb').write(src_utils)
open('explanation_models.py','wb').write(src_expl)

from model import load_model
from image_utils import imagenet_utils
from explanation_models import gradcam,gradcam_robust,gradcam_plusplus,guided_backprop


Saving model.py to model.py


Saving image_utils.py to image_utils.py


Saving explanation_models.py to explanation_models.py


In [ ]:
folder=os.path.join(wd,'ILSVRC2012_img_val/')
img_names = os.listdir(folder)
n = len(img_names)

model_name = 'vgg16'
test=load_model(model_name)
model=test.model
input_dim=test.input_dim
layer_name='block5_conv3'


g= gradcam(model,layer_name,input_dim)
heatmap_dict={}

In [ ]:
for image_name in tqdm(img_names):
    img_path=os.path.join(folder,image_name)

    img = keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    array = keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)

    preprocess_input = keras.applications.vgg16.preprocess_input

    # decode_predictions = keras.applications.vgg16.decode_predictions
    preprocessed_input=preprocess_input(array)
    top5_indices=np.argsort(-model.predict(preprocessed_input))[0][:5]
    heatmap_list=[]
    for index in top5_indices:
        heatmap_lower_dim=g.get_heatmap(preprocessed_input,index)
        heatmap = np.squeeze(heatmap_lower_dim)
        heatmap = cv2.resize(heatmap, (array.shape[2], array.shape[1]))
        heatmap_list.append([index,heatmap])
    heatmap_dict[image_name]=heatmap_list

In [ ]:
pickle.dump(heatmap_dict,open(os.path.join(wd,f'gradcam_heatmaps_{model_name}_{n}.p'),'wb'))